## Naive Bayes
Inflation VS currency exchange in "Izquierda Diario"

In [1]:
import pandas as pd
import pickle

In [2]:
data = pickle.load(open('data/df_clean_izq.pkl','rb'))

In [3]:
data.sample(5)

,topics,date,flyer,lead,headline,body,url
31,inflation,\nJueves 14 de mayo,PRECIOS,El Indec informó que el aumento de precios del...,Inflación de abril: la suba de alimentos y beb...,El Indec publicó que la inflación de abril fue...,https://www.laizquierdadiario.com/Economia/Inf...
15,inflation,\nMiércoles 21 de octubre,INDEC,Mientras los materiales para la construcción c...,"Los precios mayoristas aumentaron 3,7 % en sep...",Este miércoles el Indec publicó el índice de p...,https://www.laizquierdadiario.com/Economia/Los...
52,exchange,\nLunes 19 de octubre,TENSIÓN CAMBIARIA,A la espera de las medidas cambiarias que anun...,Dólar blue sin techo: llegó a los $ 181,"La tensión en el mercado de cambios no cede, r...",https://www.laizquierdadiario.com/Economia/Dol...
46,exchange,\nMartes 27 de octubre,TENSION CAMBIARIA,También se emitirán bonos ligados a la inflaci...,Guzmán lanza un nuevo bono atado al dólar para...,Este martes el Gobierno hará una emisión de de...,https://www.laizquierdadiario.com/Economia/Guz...
50,exchange,\nMartes 20 de octubre,TIPO DE CAMBIO,La reducción del “parking” a tres días no fue ...,"A pesar de las medidas oficiales, el dólar par...",Luego de las medidas anunciadas por el Ministe...,https://www.laizquierdadiario.com/Economia/A-p...


In [4]:
df_inflation = data.loc[data.topics == 'inflation']
df_inflation.shape

(37, 7)

In [5]:
df_exchange = data.loc[data.topics == 'exchange']
df_exchange.shape

(26, 7)

In [6]:
corpus = list(df_inflation.body.values) + list(df_exchange.body.values)
len(corpus)

63

In [7]:
target = list(df_inflation.topics.values) + list(df_exchange.topics.values)

In [8]:
data = pd.DataFrame([corpus, target]).transpose()
data

,0,1
0,La inflación no se detiene y el fin de año par...,inflation
1,"La inflación de noviembre fue del 3,2 % en com...",inflation
2,Este martes el Indec informó que la Canasta Bá...,inflation
3,El Gobierno nacional acordó con frigoríficos l...,inflation
4,Ante la crisis económica y social que afecta a...,inflation
...,...,...
58,Tras las nuevas medidas dispuestas por el Banc...,exchange
59,Tras la euforia en los “mercados” por el acuer...,exchange
60,El dólar blue aumentó casi $ 10 en una semana....,exchange
61,"El dólar blue alcanzó los $ 138, el “contado c...",exchange


### Train - Test Split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train,test,y_train,y_test = train_test_split(data.iloc[:,0], data.iloc[:,1], train_size=0.75); 

In [11]:
train.shape, test.shape

((47,), (16,))

### Tokenization + Vectorization
#### CountVectorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re

In [13]:
stopwords_sp = stopwords.words('spanish')

In [14]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode')
vectorizer.fit(list(re.sub(r'(\d|\$|\%|\+)', '', doc) for doc in corpus))

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian'

CountVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [15]:
X_train = vectorizer.transform(train);

In [16]:
X_train

<47x3006 sparse matrix of type '<class 'numpy.int64'>'
	with 6045 stored elements in Compressed Sparse Row format>

In [17]:
X_train_matriz = X_train.todense()

In [18]:
X_test = vectorizer.transform(test);

In [19]:
X_test_matriz = X_train.todense()

#### Tf IDf Vectorizer

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

In [21]:
X_train_matriz_tfidf = TfidfTransformer().fit_transform(X_train_matriz);

In [22]:
X_test_matriz_tfidf = TfidfTransformer().fit_transform(X_test_matriz);

### Multinomial Naive Bayes + CountVectorizer

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [24]:
multi_nb = MultinomialNB()

multi_nb.fit(X_train, y_train.values)

MultinomialNB()

In [25]:
y_pred = multi_nb.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
accuracy_score(y_pred, y_test.values)

1.0

In [28]:
confusion_matrix(y_pred, y_test.values)

array([[ 6,  0],
       [ 0, 10]])

### Multinomial Naive Bayes + TfIDf

In [29]:
multi_nb_2 = MultinomialNB()

multi_nb_2.fit(X_train_matriz_tfidf, y_train.values)

MultinomialNB()

In [30]:
y_pred = multi_nb_2.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
accuracy_score(y_pred, y_test.values)

1.0

In [33]:
confusion_matrix(y_pred, y_test.values)

array([[ 6,  0],
       [ 0, 10]])

### Tokenization + Vectorization (Manual)

#### CountVectorizer with pre-built tokens

In [34]:
cv_tokens = pickle.load(open('tokens/topic_tokens_izq_new.pkl', 'rb'))

In [35]:
vectorizer.fit(cv_tokens.drop('topics', axis=1))

CountVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [36]:
X_train_2 = vectorizer.transform(train)

In [37]:
X_train_matriz_2 = X_train_2.todense()

In [38]:
X_test_2 = vectorizer.transform(test)

In [39]:
X_test_matriz_2 = X_test_2.todense()

#### TF-IDF Vectorizer with pre-built tokens

In [40]:
X_train_matriz_tfidf_2 = TfidfTransformer().fit_transform(X_train_matriz_2);

In [41]:
X_test_matriz_tfidf_2 = TfidfTransformer().fit_transform(X_test_matriz_2);

### Multinomial Naive Bayes + CountVectorizer (updated tokens)

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [43]:
multi_nb = MultinomialNB()

multi_nb.fit(X_train_2, y_train.values)

MultinomialNB()

In [44]:
y_pred_2 = multi_nb.predict(X_test_2)

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(y_pred_2, y_test.values)

1.0

In [47]:
confusion_matrix(y_pred_2, y_test.values)

array([[ 6,  0],
       [ 0, 10]])

### Multinomial Naive Bayes + Tf IDf Vectorizer (updated tokens)

In [48]:
multi_nb_2 = MultinomialNB()

multi_nb_2.fit(X_train_matriz_tfidf_2, y_train.values)

MultinomialNB()

In [49]:
y_pred_2 = multi_nb_2.predict(X_test_matriz_tfidf_2)

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
accuracy_score(y_pred_2, y_test.values)

1.0

In [52]:
confusion_matrix(y_pred_2, y_test.values)

array([[ 6,  0],
       [ 0, 10]])